In [ ]:
# OpenVINO 시선 추정 모델 (gaze-estimation-adas-0002) 실습

이 노트북에서는 OpenVINO의 시선 추정 모델을 사용하여 웹캠에서 실시간으로 시선 방향을 추정해보겠습니다.

## 필요한 단계:
1. 필요한 라이브러리 설치 및 임포트
2. OpenVINO 모델 다운로드
3. 얼굴 및 랜드마크 검출 모델 준비
4. 웹캠 연결 및 실시간 추론
5. 결과 시각화

In [1]:
import sys
print(sys.executable)
!{sys.executable} -m pip install opencv-python --upgrade --quiet

/home/js/project/OpenVINO-turtleNeck/.venv/bin/python


In [9]:
# 라이브러리 임포트
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import openvino as ov
from openvino.tools import mo

In [10]:
# OpenVINO Open Model Zoo에서 시선 추정 모델 다운로드
# omz_downloader 도구를 사용하여 모델 다운로드

# 모델 저장 디렉토리 생성
model_dir = Path("./models")
model_dir.mkdir(exist_ok=True)

# 시선 추정 모델 다운로드
gaze_model_name = "gaze-estimation-adas-0002"

# omz_downloader를 사용하여 모델 다운로드
!omz_downloader --name {gaze_model_name} --output_dir ./models

print(f"모델 다운로드 완료: {gaze_model_name}")

# 다운로드된 파일 확인
gaze_model_path = model_dir / "intel" / gaze_model_name / "FP32"
if gaze_model_path.exists():
    print(f"모델 경로: {gaze_model_path}")
    print("다운로드된 파일들:")
    for file in gaze_model_path.iterdir():
        print(f"  - {file.name}")
else:
    print("모델 다운로드에 실패했습니다.")

################|| Downloading gaze-estimation-adas-0002 ||################

========== Downloading models/intel/gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.xml
Inconsistency detected by ld.so: dl-open.c: 581: dl_open_worker_begin: Assertion `_dl_debug_update (args->nsid)->r_state == RT_CONSISTENT' failed!
... 100%, 67 KB, 47 KB/s, 1 seconds passed

========== Downloading models/intel/gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.bin
... 100%, 67 KB, 47 KB/s, 1 seconds passed

========== Downloading models/intel/gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.bin
... 100%, 7352 KB, 3435 KB/s, 2 seconds passed
... 100%, 7352 KB, 3435 KB/s, 2 seconds passed

========== Downloading models/intel/gaze-estimation-adas-0002/FP16/gaze-estimation-adas-0002.xml

========== Downloading models/intel/gaze-estimation-adas-0002/FP16/gaze-estimation-adas-0002.xml
... 100%, 96 KB, 50 KB/s, 1 seconds passed

========== Downloading models/intel/gaze-estimation-adas-0002/F

In [11]:
# 시선 추정을 위해 필요한 추가 모델들 다운로드
# 1. 얼굴 검출 모델
# 2. 얼굴 랜드마크 모델 
# 3. 머리 자세 추정 모델

face_detection_model = "face-detection-adas-0001"
landmarks_model = "landmarks-regression-retail-0009" 
head_pose_model = "head-pose-estimation-adas-0001"

# 각 모델 다운로드
for model_name in [face_detection_model, landmarks_model, head_pose_model]:
    print(f"다운로드 중: {model_name}")
    !omz_downloader --name {model_name} --output_dir ./models
    print(f"완료: {model_name}")

print("\n모든 모델 다운로드 완료!")

다운로드 중: face-detection-adas-0001
################|| Downloading face-detection-adas-0001 ||################

========== Downloading models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml
################|| Downloading face-detection-adas-0001 ||################

========== Downloading models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml
... 100%, 228 KB, 323 KB/s, 0 seconds passed

========== Downloading models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.bin
... 100%, 228 KB, 323 KB/s, 0 seconds passed

========== Downloading models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.bin
... 100%, 4113 KB, 4327 KB/s, 0 seconds passed

========== Downloading models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml
... 100%, 4113 KB, 4327 KB/s, 0 seconds passed

========== Downloading models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml
... 100%, 304 KB, 79 KB/s, 3 seconds passed

========== D

In [12]:
# 다운로드된 모델 파일들 확인 및 설정
# OpenVINO 모델들은 .xml (모델 구조)과 .bin (가중치) 파일로 구성됨

# 모델 경로들 설정
models_base = Path("./models/intel")

gaze_model_xml = models_base / "gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.xml"
face_detection_xml = models_base / "face-detection-adas-0001/FP32/face-detection-adas-0001.xml"
landmarks_xml = models_base / "landmarks-regression-retail-0009/FP32/landmarks-regression-retail-0009.xml"
head_pose_xml = models_base / "head-pose-estimation-adas-0001/FP32/head-pose-estimation-adas-0001.xml"

# 모델 파일 존재 확인
model_files = {
    "시선 추정": gaze_model_xml,
    "얼굴 검출": face_detection_xml,
    "랜드마크": landmarks_xml,
    "머리 자세": head_pose_xml
}

print("모델 파일 확인:")
for name, path in model_files.items():
    if path.exists():
        print(f"✓ {name}: {path}")
    else:
        print(f"✗ {name}: {path} (파일이 없습니다)")

# OpenVINO 코어 초기화
core = ov.Core()
print(f"\nOpenVINO 버전: {core.get_versions(core.available_devices[0])}")
print(f"사용 가능한 디바이스: {core.available_devices}")

모델 파일 확인:
✓ 시선 추정: models/intel/gaze-estimation-adas-0002/FP32/gaze-estimation-adas-0002.xml
✓ 얼굴 검출: models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml
✓ 랜드마크: models/intel/landmarks-regression-retail-0009/FP32/landmarks-regression-retail-0009.xml
✓ 머리 자세: models/intel/head-pose-estimation-adas-0001/FP32/head-pose-estimation-adas-0001.xml

OpenVINO 버전: {'CPU': <Version: 2024.6.0-17404-4c0f47d2335-releases/2024/6 openvino_intel_cpu_plugin>}
사용 가능한 디바이스: ['CPU']


In [13]:
# 모델들을 메모리에 로드하기
device = "CPU"  # GPU가 있으면 "GPU"로 변경 가능

try:
    # 모델 읽기 및 컴파일
    gaze_model = core.read_model(model=gaze_model_xml)
    face_model = core.read_model(model=face_detection_xml)
    landmarks_model = core.read_model(model=landmarks_xml)
    head_pose_model = core.read_model(model=head_pose_xml)
    
    # 모델 컴파일 (추론 준비)
    gaze_compiled = core.compile_model(model=gaze_model, device_name=device)
    face_compiled = core.compile_model(model=face_model, device_name=device)
    landmarks_compiled = core.compile_model(model=landmarks_model, device_name=device)
    head_pose_compiled = core.compile_model(model=head_pose_model, device_name=device)
    
    print("✓ 모든 모델이 성공적으로 로드되었습니다!")
    
    # 모델 입출력 정보 확인
    print("\n=== 시선 추정 모델 정보 ===")
    print(f"입력: {[input.any_name for input in gaze_model.inputs]}")
    print(f"출력: {[output.any_name for output in gaze_model.outputs]}")
    
    print("\n=== 얼굴 검출 모델 정보 ===")
    print(f"입력: {[input.any_name for input in face_model.inputs]}")
    print(f"출력: {[output.any_name for output in face_model.outputs]}")
    
except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    print("모델 파일들이 올바르게 다운로드되었는지 확인해주세요.")

✓ 모든 모델이 성공적으로 로드되었습니다!

=== 시선 추정 모델 정보 ===
입력: ['left_eye_image', 'right_eye_image', 'head_pose_angles']
출력: ['gaze_vector']

=== 얼굴 검출 모델 정보 ===
입력: ['data']
출력: ['detection_out']


In [14]:
## 웹캠 실시간 시선 추정 및 시각화
# OpenCV로 웹캠을 열고, 얼굴/눈/머리포즈/시선 추정 결과를 실시간으로 시각화합니다.

import cv2
import numpy as np

# 얼굴 검출, 랜드마크, 머리포즈, 시선 추정 모델의 컴파일된 객체를 사용합니다.
# face_compiled, landmarks_compiled, head_pose_compiled, gaze_compiled

def draw_gaze_vector(frame, eye_center, gaze_vector, scale=60, color=(0,0,255)):
    """시선 벡터를 프레임에 그려줍니다."""
    x, y = int(eye_center[0]), int(eye_center[1])
    dx, dy = int(gaze_vector[0]*scale), int(-gaze_vector[1]*scale)
    cv2.arrowedLine(frame, (x, y), (x+dx, y+dy), color, 2, tipLength=0.3)

cap = cv2.VideoCapture(0)
w, h = 640, 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, w)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, h)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. 얼굴 검출
    input_blob = cv2.resize(frame, (672, 384))
    input_blob = input_blob.transpose(2,0,1)[np.newaxis, ...]
    input_blob = input_blob.astype(np.float32)
    face_results = face_compiled([input_blob])[face_compiled.outputs[0]]
    faces = face_results[0][0]
    for det in faces:
        conf = det[2]
        if conf < 0.5:
            continue
        xmin, ymin, xmax, ymax = (det[3:7] * np.array([w, h, w, h])).astype(int)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0,255,0), 2)

        # 2. 랜드마크 추정 (눈 위치)
        face_img = frame[ymin:ymax, xmin:xmax]
        if face_img.size == 0:
            continue
        lm_input = cv2.resize(face_img, (48, 48)).transpose(2,0,1)[np.newaxis, ...].astype(np.float32)
        lm_results = landmarks_compiled([lm_input])[landmarks_compiled.outputs[0]]
        lm = lm_results.reshape(-1,2)
        left_eye = (int(lm[0][0]*(xmax-xmin))+xmin, int(lm[0][1]*(ymax-ymin))+ymin)
        right_eye = (int(lm[1][0]*(xmax-xmin))+xmin, int(lm[1][1]*(ymax-ymin))+ymin)
        cv2.circle(frame, left_eye, 3, (255,0,0), -1)
        cv2.circle(frame, right_eye, 3, (0,0,255), -1)

        # 3. 머리포즈 추정
        hp_input = cv2.resize(face_img, (60, 60)).transpose(2,0,1)[np.newaxis, ...].astype(np.float32)
        hp_results = head_pose_compiled([hp_input])
        head_pose = [hp_results[o][0][0] for o in head_pose_compiled.outputs]

        # 4. 시선 추정
        # 눈 crop
        eye_size = 60
        def crop_eye(center):
            x, y = center
            x1, y1 = max(0, x-eye_size//2), max(0, y-eye_size//2)
            x2, y2 = min(w, x+eye_size//2), min(h, y+eye_size//2)
            eye = frame[y1:y2, x1:x2]
            if eye.shape[0]!=eye_size or eye.shape[1]!=eye_size:
                eye = cv2.resize(eye, (eye_size, eye_size))
            return eye.transpose(2,0,1)[np.newaxis, ...].astype(np.float32)
        left_eye_img = crop_eye(left_eye)
        right_eye_img = crop_eye(right_eye)
        head_pose_np = np.array(head_pose, dtype=np.float32).reshape(1,3)
        gaze_result = gaze_compiled([left_eye_img, right_eye_img, head_pose_np])
        gaze_vector = gaze_result[gaze_compiled.outputs[0]][0]

        # 5. 시선 벡터 시각화
        eye_center = ((left_eye[0]+right_eye[0])//2, (left_eye[1]+right_eye[1])//2)
        draw_gaze_vector(frame, eye_center, gaze_vector)

    cv2.imshow("Gaze Estimation", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()